In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp
import math
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append("./class")
#from Flow_class import Flow

In [3]:
Nx=20
Ny=20

In [4]:
gb, domain = pp.grid_buckets_2d.single_horizontal([Nx, Ny], x_endpoints=[0.2, 0.8], simplex=False)

In [5]:
parameter={"aperture":1e-3,
          "fracture_perm":1e-5,
          "perm":1,
          "f_lambda":lambda x,y,z:1e-2,
          "bc_value":[0.0,0.0],
          "bc_type":['dir','dir'],
          "method":"Tpfa"}

In [6]:
#flow=Flow(gb,domain,parameter)

In [7]:
#flow.set_data()
#flow.discretize()
#flow.solve()
#flow.plot_pressure()

In [8]:
# Thickness of fracture
aperture = 1e-3
fracture_perm = 1e-5
data_key="flow"
for g, d in gb:
    # The concept of specific volumes accounts for the thickness
    # of the fracture, which is collapsed in the mixed-dimensional
    # model.
    specific_volumes = np.power(aperture, gb.dim_max()-g.dim)#è legato al fatto che la dimensione della fratttura 
    #viene ridotta sempre di un unità (cioè ha un apertura diversa da zero quindi dovrebbe essere bidimensionale, 
    #ma viene ridotta comunque di una dimensione e per far ciò si modificano le sue proprietà fisiche, 
    #ovvero vengono "concentrate")
    # Permeability
    k = np.ones(g.num_cells) * specific_volumes#è la kx e basta per la frattura
    if g.dim < gb.dim_max():#la g è quella della frattura?
        k *= fracture_perm
    perm = pp.SecondOrderTensor(k)   
    #perm=pp.SecondOrderTensor(kxx=k,kyy=k,kzz=1)

    # Unitary scalar source already integrated in each cell
    f = 1e-2* g.cell_volumes * specific_volumes 

    # Boundary conditions
    b_faces = g.tags['domain_boundary_faces'].nonzero()[0]
    bc = pp.BoundaryCondition(g, b_faces, ['dir']*b_faces.size)
    bc_val = np.zeros(g.num_faces)
    bc_val[b_faces] = g.face_centers[1, b_faces]
    
    parameters = {"second_order_tensor": perm, "source": f, "bc": bc, "bc_values": bc_val}
    pp.initialize_data(g, d, data_key, parameters)

for e, d in gb.edges():
        gl, _ = gb.nodes_of_edge(e)
        mg = d["mortar_grid"]
        # Division through aperture/2 may be thought of as taking the gradient, i.e.
        # dividing by the distance from the matrix to the center of the fracture.
        kn = fracture_perm / (aperture/2)
        pp.initialize_data(mg, d, data_key, {"normal_diffusivity": kn})

In [9]:
flow_discretization = pp.Tpfa(data_key)
source_discretization = pp.ScalarSource(data_key)
for g, d in gb:
        d[pp.PRIMARY_VARIABLES] = {"pressure": {"cells": 1}}#gradi libertà, per misti "faces":1
        d[pp.DISCRETIZATION] = {"pressure": {"diffusive": flow_discretization,
                                             "source": source_discretization}}
        
flow_coupling_discretization = pp.RobinCoupling(data_key, flow_discretization)        
for e, d in gb.edges():
    g1, g2 = gb.nodes_of_edge(e)
    d[pp.PRIMARY_VARIABLES] = {"mortar_flux": {"cells": 1}}
    d[pp.COUPLING_DISCRETIZATION] = {
        "lambda": {
            g1: ("pressure", "diffusive"),
            g2: ("pressure", "diffusive"),
            e: ("mortar_flux", flow_coupling_discretization),
        }
    }


In [10]:
assembler = pp.Assembler(gb)
assembler.discretize()
A, b = assembler.assemble_matrix_rhs(add_matrices=False)
print(A)
#solution = sps.linalg.spsolve(A, b)
#assembler.distribute_variable(solution)

{'source_pressure': <436x436 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>, 'diffusive_pressure': <436x436 sparse matrix of type '<class 'numpy.float64'>'
	with 1930 stored elements in Compressed Sparse Row format>, 'lambda_mortar_flux_pressure_pressure': <436x436 sparse matrix of type '<class 'numpy.float64'>'
	with 120 stored elements in Compressed Sparse Row format>}


In [11]:
#pp.plot_grid(gb, "pressure",figsize=(15, 12))